In [1]:
import os
os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    local_data_file: Path


In [3]:
from src.pixel_predictor.constants import *
from src.pixel_predictor.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
                 ):
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
          config = self.config.data_ingestion

          create_directories([config.root_dir])

          data_ingestion_config = DataIngestionConfig(
                root_dir = config.root_dir,    
                local_data_file = config.local_data_file,
          )
          return data_ingestion_config

In [5]:
from pixel_predictor import logger

In [6]:
import numpy as np
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def create_file(self,num_images=10000)-> str:
        '''
        This will create files
        data_file_path:str
        make zip from created files and store 
        '''
        try:
            images = []
            labels = []
            for _ in range(num_images):
                image = np.zeros((50, 50), dtype=np.float32)
                x, y = np.random.randint(0, 50, size=2)
                image[x, y] = 255
                images.append(image)
                labels.append([x, y])
            data_file_path = self.config.local_data_file
           # os.makedirs(data_file_path,exist_ok=True)
            np.savez_compressed(data_file_path,X=np.array(images), y=np.array(labels))
            logger.info(f"Created image data and stored into file {data_file_path}")

        except Exception as e:
            raise e

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.create_file(num_images=1000)
except Exception as e:
    raise e

[2024-05-21 16:20:24,519: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-21 16:20:24,521: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-21 16:20:24,522: INFO: common: created directory at: artifacts]
[2024-05-21 16:20:24,523: INFO: common: created directory at: artifacts/data_ingestion]
[2024-05-21 16:20:24,616: INFO: 114488486: Created image data and stored into file artifacts/data_ingestion/data.npz]
